In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [ ]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso

In [ ]:
test_2.head()
test_2.shape

In [ ]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

In [ ]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 
print(test_2['patho_niv1'].unique())

['Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitements du risque vasculaire (hors pathologies)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'
 'Traitements psychotropes (hors pathologies)'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies psychiatriques' nan
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)' 'Diabète'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Insuffisance rénale chronique terminale']


In [ ]:
data_gen = test_2

In [ ]:
print(test_2.shape, data_gen.shape)

In [ ]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

In [ ]:
print(data_gen.shape, data_gen['patho_niv1'].isna().sum())

In [ ]:
#Creation d une colonne individus
ind = []
for i in range (1,4057202) : 
    ind.append(i)

In [ ]:
ind = pd.DataFrame(ind)
ind = ind.set_axis(['ind'], axis='columns')

In [ ]:
data_gen_1 = pd.concat([ind, data_gen], axis = 1)

In [ ]:
data_gen_1.head()

In [ ]:
data_gen = data_gen_1

In [ ]:
#tentative de compréhension de ce que la variable npop représente
#b = data_gen['npop'].unique()
#b = pd.DataFrame(b)

In [ ]:
#b = b.dropna()

In [ ]:
#b = b.set_axis(['test'], axis = 'columns')

In [ ]:
#b.head()

In [ ]:
#sum(b['test'])

In [ ]:
#resultat peu concluant

In [ ]:
#creation d indicatrices pour futur clustering

In [ ]:
liste_diff_patho3 = list(liste_diff_patho3)

In [ ]:
#premier code pour creation d indicatrice

In [ ]:
temporaire = []
for i in data_gen['patho_niv3'] : 
    if i == 'Syndrome coronaire aigu' :
        temporaire.append(1)
    else : 
        temporaire.append(0)

data_gen['Syndrome_coronaire_aigu_indicatrice'] = temporaire
data_gen = data_gen.drop(columns = ['Syndrome_coronaire_aigu_indicatrice'])
data_gen.head()

In [ ]:
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    data_gen[f"{nom_de_maladie}_ind"] = temporaire

In [ ]:
for i in liste_diff_patho3 : 
    creation_indicatrice(colonne_de_depart = data_gen['patho_niv3'], nom_de_maladie = i)

In [ ]:
data_gen.shape

In [ ]:
#perte d un grand nombre de lignes 
#pb a regler
#pb regle, venait du test pour essayer de comprendre la variable npop

In [ ]:
liste_var = data_gen.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)


In [30]:
indicatrice_patho_niv3 = liste_bis

In [ ]:
indicatrice_patho_niv3 = liste_bis 
print(indicatrice_patho_niv3)

In [ ]:
liste_auxil = data_gen['cla_age_5'].unique()
print(liste_auxil)

#afin de pouvoir clusteriser par la suite, nous allons creer des classes d age qui sont "discretes"
#nous faisons ce choix car les classes d age sont relativement homogene et en prenant la moyenne des deux 
#bornes pour les classes, nous gardons l idee d une distance d age qui pourrait etre utile par la suite

inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)


classe_age = []
for i in data_gen['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)

#pb a regler, il manque un truc pour eviter que ca bug fort
#probablement type a regler ou categorie a revoir
#code peu ergonomique mais solution la plus sure, 
#toutes les autres tentatives en posant des conditions sur le type de i 
#n ont pas ete solde par une reussite
#on a print nombre de valeurs de classe_age et le code fonctionne

data_gen['classe_age'] = classe_age

data_patho_niv3 = data_gen

In [31]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
liste_auxil = data_gen['cla_age_5'].unique()

In [ ]:
data_gen = data_gen.drop(indicatrice_patho_niv3, axis = 1)

In [ ]:
#on cree un nouveau dataframe pour pouvoir faire des manipulations plus rapides sur le data_gen

In [ ]:
data_gen.head()

In [ ]:
df_model = data_gen[['ind','classe_age' , 'region', 'dept']]

In [ ]:
df_model_niv3 = pd.concat([df_model, data_patho_niv3[indicatrice_patho_niv3]], axis = 1)

In [ ]:
#truc bizarre a reverifier mais il se peut qu'il y ait eu un petit bug avec une colonne nan_ind

In [ ]:
#l'autre base pour la pollution ne prend pas les départements d'outre-mer en compte
#il est donc probable que l'on soit contraints de les exclure de l'analyse 


In [ ]:
data_pollution = pd.read_csv("base_pollution.csv")

In [ ]:
data_pollution.head()

In [ ]:
#df_model_test = pd.merge(df_model, data_pollution, how = 'left', left_on='dept', right_on='DEP')

In [ ]:
#decision prise apres cette ligne de reduire 
#considerablement la taille de la base sans quoi aucun calcul ne serait possible
#j'ai essaye d'augmenter drastiquement les ressources disponibles pour 
#pouvoir ne serait-ce que merge et pour le clustering, 
#le calcul semble impossible

In [ ]:
#data_sans_na = data_gen[data_gen['patho_niv1'].apply(lambda s: s != 'Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire')]

In [ ]:
data_sans_na = data_gen.dropna()

In [ ]:
df_model_reduced = data_sans_na[['ind','classe_age' , 'region', 'dept']]

In [ ]:
def creation_indicatrice_modif (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    data_sans_na.loc[:, f"{nom_de_maladie}_ind"] = temporaire
#derniere ligne donnee par chatgpt parce qu'il y avait une erreur
#car je n'utilisais pas le .loc mais mettais juste :
#data_sans_na[f"{nom_de_maladie}_ind"], mais apparemment 
#cela aurait pu faire un comportement etrange et donc pour modifier 
#"en vrai" le nouveau subset il fallait le traiter comme ca,
#ce qui semble marcher

for i in liste_diff_patho3 : 
    creation_indicatrice_modif(colonne_de_depart = data_sans_na.loc[:,'patho_niv3'], nom_de_maladie = i)

In [ ]:
data_sans_na.head()

In [ ]:
df_model_reduced.shape

In [ ]:
df_model_reduced.head()

In [1]:
#meme en ayant divise par deux la taille de la base,
#augmente de facon considerable les ressources disponibles,
#efface tous les outputs des cellules
#ca ne marche pas 
#donc il faudra faire des regroupements
#vient la question : lesquels